<a href="https://colab.research.google.com/github/jinsusong/study-Kaggle/blob/main/Classification_Spooky_Author_Identification_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle Spooky Author Identification 데이터 불러오기 

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

In [ ]:
#!pip install --upgrade --force-reinstall --no-deps kaggle

In [ ]:

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Permission Warning 방지
!chmod 600 ~/.kaggle/kaggle.json
!pwd

In [ ]:
!kaggle competitions download -c spooky-author-identification

In [ ]:
!ls -al

# 데이터 압축 해제

In [ ]:
!mkdir data
!unzip sample_submission.zip -d data/
!unzip test.zip -d data/
!unzip train.zip -d data/

# 데이터 확인하기 

In [ ]:
import pandas as pd 

label_cols = ['author']
train = pd.read_csv("data/train.csv")
print(len(train))
#len_eap = train['author'] == 'EAP'
#len_eap
train.head()



In [ ]:

test = pd.read_csv("data/test.csv")
test.head()


In [ ]:
print(train.author.value_counts())
train.head()

#Preprocess Data 

In [ ]:

from sklearn import preprocessing

#data preprocssing
encoder = preprocessing.LabelEncoder()
train['author'] = encoder.fit_transform(train["author"].values)

print(len(train))

In [ ]:
type(train)

In [ ]:
from sklearn.model_selection import train_test_split
X = list(train["text"])
y = list(train["author"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)

#X_train, X_test, y_train, y_test = train_test_split(bert_train['text'], bert_train['target'], test_size=0.25, random_state=42)

In [ ]:
print(len(X_train))
print(len(y_train))
X_train[:10]
y_train[:10]


In [ ]:
type(X_train)
type(y_train)

In [ ]:
!pip install transformers

# 버트 토크나이저 로드 

In [ ]:
from transformers import AutoTokenizer

bert_model_name = 'bert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(bert_model_name)


#Tokenized

In [ ]:
train_encodings = dict(tokenizer(X_train, padding=True, truncation=True, max_length=512,add_special_tokens=True, return_attention_mask=True))
val_encodings = dict(tokenizer(X_val, padding=True, truncation=True, max_length=512))
len(train_encodings)

#데이터 셋으로 변환

In [ ]:
import tensorflow as tf
batch_size = 32
train_data = tf.data.Dataset.from_tensor_slices((train_encodings,y_train)).batch(batch_size).prefetch(1)



In [ ]:
train_data

In [ ]:

len(train_data)
i = 0
for element in train_data:
  i += 1
  print(element)
  if i >= 1 : break

# Training the Model 

In [ ]:
# from transformers import TFBertForMultipleChoice

# model = TFBertForMultipleChoice.from_pretrained(bert_model_name)


from transformers import TFBertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = TFBertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 3, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

In [ ]:
# train_data 
# train_val_data 
# y_data 
# y_val_data 

# loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# # model.compile(optimizer='adam',loss=loss)
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
            #   loss=tf.keras.losses.sparse_categorical_crossentropy(from_logits=True), 
            #   metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

learning_rate = 2e-5

# we will do just 1 epoch for illustration, though multiple epochs might be better as long as we will not overfit the model
number_of_epochs = 1

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)

# we do not have one-hot vectors, we can use sparce categorical cross entropy and accuracy
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [ ]:
model.summary()

In [ ]:
# 모델의 구조 확인하기 
tf.keras.utils.plot_model(model)

In [ ]:
type(y_train)
type(train_data)

In [ ]:
model.fit(x=train_data,y=None , epochs=1, batch_size=32)
#model.fit(train_data, validation_data=train_val_data, epochs=1)
# test_encodings["input_ids"]
# y_data

In [ ]:
!pwd

In [ ]:
#model.save("SpookyAuthor.h5")

tf.keras.models.save_model(model, "/content/spookyModel") 